In [3]:
import copy
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
s = 2
rho = 0.05
z1 = .1
z2 = .2
z = np.array([z1,z2])
la1 = 0.6
la2 = 0.8
la = np.array([la1,la2])

Ir = 40
r0 = 0.03
rmin = 0.01
rmax = 0.04

I = 1000
amin = -0.15
amax = 4
a=np.linspace(amin, amax, I)
da=(amax-amin)/(I-1)

aa=np.array([a,a]).T
zz=np.ones((I,1))*z

maxit = 100
crit = 10**(-6)
Delta = 100

dVf = np.zeros((I,2))
dVb = np.zeros((I,2))
c=np.zeros((I,2))

Aswitch1 = scipy.sparse.hstack([-scipy.sparse.eye(I)*la1, scipy.sparse.eye(I)*la1])
Aswitch2 = scipy.sparse.hstack([scipy.sparse.eye(I)*la2, -scipy.sparse.eye(I)*la2])
Aswitch = scipy.sparse.vstack([Aswitch1, Aswitch2])

crit_S = 10**(-8)

# Inintial Guess
r = r0
v01 = (z[0]+r*a)**(1-s)/(1-s)/rho
v02 = (z[1]+r*a)**(1-s)/(1-s)/rho
v0 = np.array([v01,v02]).T
v = copy.deepcopy(v0)

终止时刻 

In [4]:
S = np.zeros(Ir)
dist = np.zeros(maxit)
for ir in range(Ir):
    if ir>0:
        v = copy.deepcopy(V)
    elif ir == 0:
        v=copy.deepcopy(v0)
    for n in range(maxit):
        V = copy.deepcopy(v)
        ## forward difference method eq.(8)
        dVf[0:I-1,:] = (V[1:I,:]-V[0:I-1,:])/da
        dVf[I-1,:] = 1 ## never be used, but set 1 make sure there is no exception
        ## backward difference method eq.(8)
        dVb[1:I,:] = (V[1:I,:]-V[0:I-1,:])/da
        dVb[0,:] = (z+r*amin)**(-s)
        ## consumption and savings with forward difference     u=x^(1-s)/1-s  => (u')^-1: x^(-1/s) 
        cf = dVf**(-1/s)
        sf = zz + r*aa - cf
        ## consumption and savings with backward difference
        cb = dVb**(-1/s)
        sb = zz + r*aa - cb
        ## consumption and derivative of value function at steady state
        c0 = zz + r*aa
        dV0 = c0**(-s)
        ## the indicator of upwind scheme 
        If = np.where(sf>0, 1, 0)
        Ib = np.where(sb<0, 1, 0)
        I0 = (1-If-Ib)     ## sijF<0<sijB
        dV_Upwind = dVf*If + dVb*Ib + dV0*I0
        ## compute c and u
        c = dV_Upwind**(-1/s)
        u = c**(1-s)/(1-s)
        ## CONSTRUCT MATRIX
        X = -np.minimum(sb,0)/da
        Y = -np.maximum(sf,0)/da + np.minimum(sb,0)/da
        Z = np.maximum(sf,0)/da
        A1 = scipy.sparse.spdiags(Y[:,0],0,I,I)+\
            scipy.sparse.spdiags(X[1:,0],-1,I,I)+\
            scipy.sparse.spdiags(np.concatenate([np.array([0]),Z[:,0]]),1,I,I)
        A2 = scipy.sparse.spdiags(Y[:,1],0,I,I)+\
            scipy.sparse.spdiags(X[1:,1],-1,I,I)+\
                scipy.sparse.spdiags(np.concatenate([np.array([0]),Z[:,1]]),1,I,I)
        A = scipy.sparse.vstack([\
            scipy.sparse.hstack([A1, scipy.sparse.csc_matrix((I,I))]),\
                  scipy.sparse.hstack([scipy.sparse.csc_matrix((I,I)), A2])]) + Aswitch
        B = (rho + 1/Delta)*scipy.sparse.eye(2*I) - A
        u_stacked = np.vstack((u[:,0].reshape((I,1)), u[:,1].reshape((I,1))))
        V_stacked = np.vstack([V[:,0].reshape((I,1)), V[:,1].reshape((I,1))])
        b = u_stacked + V_stacked/Delta
        V_stacked = scipy.sparse.linalg.spsolve(B, b)
        V[:,0] = np.squeeze(V_stacked[0:I])
        V[:,1] = np.squeeze(V_stacked[I:])
        Vchange = V - v
        v=copy.deepcopy(V)
        dist[n] = np.max(np.abs(Vchange))
        if dist[n] < crit:
            break
    print("Value function is converged. Iteration is:", n)
    AT = A.T 
    b = np.zeros((2*I,1))
    """ 
    consider page 9. We need to fix g[i,j]=0.1 (for an arbitrary (i,j)).
    Fixing it can be solved by setting one element from the 0 of eq(20) ATg=0 to be 0.1
    For convenience, we set g[0,j] = 0.1 
    """
    i_fix = 0
    b[i_fix] = 0.1
    row = np.zeros(I*2)
    row[i_fix] = 1
    AT[i_fix,:] = row

    """ 
    solve eq(20) ATg=0
    """
    gg = scipy.sparse.linalg.spsolve(AT, b)
    gg_sum = gg.T @ np.ones((2*I,1)) * da
    gg = gg/gg_sum
    g1 = gg[0:I]
    g2 = gg[I:]

    ## compute S(r)
    S[ir] = g1@a*da + g2@a*da
    ## UPDATE INTEREST RATE
    if S[ir] > crit_S:
        print('Excess Supply')
        rmax = r
        r = 0.5 * (r + rmax)
    elif S[ir] < -crit_S:
        print('Excess Demand')
        rmin = r
        r = 0.5 * (r + rmin)


Value function is converged. Iteration is: 15
